# 컨텐츠 기반 필터링 실습

- TMDB 5000 movie 데이터셋

In [40]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data/tmdb_data')

In [41]:
movies = pd.read_csv('tmdb_5000_movies.csv', encoding='utf-8')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [42]:
# 컨텐츠를 추천하는 시스템을 만들기 위해서
# 해당 영화들의 유사도 비교할 항목들만 추출
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]
movies_df.shape

(4803, 8)

In [43]:
# 출력되는 칼럼의 넓이 조절 가능
pd.options.display.max_colwidth = 100
# 영화의 장르와 키워드 하나만 추출해보기
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [44]:
# 장르와 키워드 값들이 딕셔너리가 들어있는 리스트로 '문자열'상태로 되어있음
# 이것을 바꾸어주어야 함!
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


**텍스트를 파이썬 자료구조 형태로 바꾸어주어야 함!**<br>
**파이썬의 추상구문 트리인 ast 클래스의 literal_eval 매서드 사용**
- literal_eval : 제공된 문자열이나 노드는 다음과 같은 파이썬 리터럴 구조로만 구성될 수 있습니다: 문자열, 바이트열, 숫자, 튜플, 리스트, 딕셔너리, 집합, 불리언 및 None

- 위 함수 수행시 파이썬 자료구조를 기준으로해서 하나의 객체씩 할당됨

In [45]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [46]:
# genres라는 dict형태에서 'name' key의 value들만 가져오기
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
# keywords도 동일하게 수행
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])

In [47]:
# 리스트에 담겨있는 genres, keywords를 공백 기준으로 문자열로 붙이기
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x))
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,keywords_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony society space travel futuristic romance space alien ...


In [52]:
# 모든 변수가 텍스트로 이루어졌으니 Countvectorizer로 feature 벡터화시키기
# 경우에 따라 추가적으로 Lemmatization 추가해 토큰화 전용 함수 추가도 가능!

from sklearn.feature_extraction.text import CountVectorizer
cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
genres_vect = cnt_vect.fit_transform(movies_df['genres_literal'])
keywords_vect = cnt_vect.fit_transform(movies_df['keywords_literal'])
print('장르 벡터화 shape: ', genres_vect.shape)
print('키워드 벡터화 shape: ', keywords_vect.shape)

장르 벡터화 shape:  (4803, 276)
키워드 벡터화 shape:  (4803, 41554)


In [54]:
# 장르에 따른 영화별 코사인 유사도 추출
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genres_vect, genres_vect)
print(genre_sim.shape)
print(genre_sim[:3])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [57]:
# argsort를 이용해서 유사도가 높은 영화들의 index 추출
genre_sim_idx = genre_sim.argsort()[::-1]
# 하나만 추출
print(genre_sim_idx[:1])

[[   0 3141 3140 ... 4521 4710 4802]]


In [94]:
# 특정 영화를 기준으로 선정해서 유사도가 높은 영화를 반환하는 함수 생성
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    # 비교기준으로 선정할 특정 영화 선정
    title_movie = df[df['title'] == title_name]
    
    # 비교기준 영화의 index.values를 할당해 유사도 행렬에서 비교기준 영화에 해당하는 유사도 행렬값을 찾자!
    title_movie_idx = title_movie.index.values
    # 모든 영화끼리의 유사도 행렬에서 비교기준 영화에 대한 유사도행렬 찾아서 할당
    top_sim_idx = sorted_idx[title_movie_idx, :top_n]
    print(top_sim_idx)
    
    # top_sim_idx는 2차원 array이기 때문에 1차원 array로 변경
    # 왜냐하면 top_sim_idx로 movies_df의 index값으로 넣어서 유사한 영화추출할 것이기 때문
    top_sim_idx = top_sim_idx.reshape(-1,)
    similar_movie = df.iloc[top_sim_idx]
    
    return similar_movie

In [ ]:
# 어벤져스 영화와 관련된 영화들 추출 

In [101]:
similar_movies = find_sim_movie(movies_df, genre_sim_idx, 'The Avengers')
similar_movies[['title','vote_average','vote_count']]

[[   0 2743 2749 2750 2751 2752 2755 2757 2760 2764]]


,title,vote_average,vote_count
0,Avatar,7.2,11800
2743,The Butterfly Effect,7.3,2060
2749,Child's Play 2,5.8,308
2750,No Good Deed,5.6,181
2751,The Mist,6.7,1399
2752,Ex Machina,7.6,4737
2755,Earth to Echo,5.7,290
2757,Letters from Iwo Jima,7.2,541
2760,Room,8.1,2757
2764,Light It Up,6.6,7


## 가중 평점을 활용해서 추가적인 변수를 넣기

- v : 개별 영화에 평점을 투표한 횟수
- m : 평점을 부여하기 위한 ``최소`` 투표 횟수
- R : 개별 영화에 대한 평균 평점
- C : 전체 영화에 대한 평균 평점

In [106]:
C = movies_df['vote_average'].mean()

# 투표횟수 중 60%이상의 횟수에 달하는 숫자
# 예를들어 총 투표횟수가 100과 1일때 m값은 매우 달라진다.
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R) + ( (m/(m+v)) * C)

In [109]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [110]:
movies_df[['title','weighted_vote']].sort_values(by=['weighted_vote'])[:10]

,title,weighted_vote
1652,Dragonball Evolution,4.320022
210,Batman & Robin,4.591725
3746,The Boy Next Door,4.629738
242,Fantastic Four,4.636554
2122,Epic Movie,4.737894
303,Catwoman,4.794536
3843,Sharknado,4.802791
480,Battlefield Earth,4.830969
499,Jack and Jill,4.857033
2320,Left Behind,4.861876


In [121]:
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_idx = title_movie.index.values
    
    similar_idx = sorted_idx[title_idx, :(top_n*2)]
    similar_idx = similar_idx.reshape(-1,)
    
    #자기 자신 영화 제외, boolean index기법 사용!
    similar_idx = similar_idx[similar_idx != title_idx]
    
    return df.iloc[similar_idx].sort_values(by=['weighted_vote'],
                                           ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_idx,
                               'The Avengers')
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2760,Room,8.1,7.862312
2752,Ex Machina,7.6,7.490704
0,Avatar,7.2,7.166301
2743,The Butterfly Effect,7.3,7.116008
2757,Letters from Iwo Jima,7.2,6.749914
2751,The Mist,6.7,6.572814
2783,The Call,6.6,6.484465
2787,Quest for Fire,7.1,6.321415
2742,The Shallows,6.2,6.179394
2778,Infamous,6.4,6.135104
